# Building a GenAI RAG application with Feature Store and BigQuery

## Overview
This notebook guides you through building a low-latency vector search system for your GenAI application using Vertex AI Feature Store. We'll leverage the [Vertex Feature Store Langchain integration]([link to integration]) to streamline this process.

Feature Store seamlessly integrates with BigQuery, providing a unified data storage and flexible vector search options:

- **BigQuery Vector Search**: Ideal for batch retrieval and prototyping, as it requires no infrastructure setup.
- **Feature Store Online Store**: Enables low-latency retrieval with manual or scheduled data sync. Perfect for production-ready user-facing GenAI applications.
![Image notebook journey](diagram_journey.png)


# Setup


### Install libraries

In [ ]:
!pip install langchain-google-vertexai pypdf==4.2.0 langchain pyarrow==16.0.0 db-dtypes==1.2.0 --upgrade

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
# from google.colab import auth

# auth.authenticate_user()

### Import libraries

In [ ]:
# import sys
# sys.path.append("../")

In [ ]:
%load_ext autoreload
%autoreload 2
from langchain_google_community import VertexFSVectorStore
from langchain_google_community import BigQueryVectorStore

### Define environment variables

In [ ]:
PROJECT_ID = "cloud-llm-preview2"
DATASET = "vertex_documentation_new1"
TABLE = "mytest4"
REGION = "europe-west4"

# Add documents to `VertexAIFeatureStore`

This step ingests and parse PDF documents, split them, generate embeddings and add the embeddings to the vector store. The document corpus used as dataset is a collection of owners car manual.

**Summary steps**
- Create text embeddings: LangChain `VertexAIEmbeddings`
- Ingest PDF files: LangChain `PyPDFLoader`
- Chunk documents: LangChain `TextSplitter`
- Create Vector Store: LangChain  `VertexAIFeatureStore` 

### Create the VertexAI Embedding model

In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_community.vectorstores import BigQueryVectorSearch

embedding_model = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)

### Ingest PDF file

The document is hosted on Cloud Storage bucket (at `gs://github-repo/generative-ai/sample-apps/fixmycar/cymbal-starlight-2024.pdf`) and LangChain provides a convenient document loader [`PyPDFLoader`](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf/) to load documents from pdfs.


In [ ]:
GCS_BUCKET_DOCS = (
    "github-repo/generative-ai/sample-apps/fixmycar"  # @param {type: "string"}
)

# Copy the file to the current path
!gsutil cp "gs://$GCS_BUCKET_DOCS/*.pdf" .

In [ ]:
# Ingest PDF files
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("cymbal-starlight-2024.pdf")
documents = loader.load()


# Add document name and source to the metadata
for document in documents:
    doc_md = document.metadata
    document_name = doc_md["source"].split("/")[-1]
    # derive doc source from Document loader
    doc_source_prefix = "/".join(GCS_BUCKET_DOCS.split("/")[:3])
    doc_source_suffix = "/".join(doc_md["source"].split("/")[4:-1])
    source = f"{doc_source_prefix}/{doc_source_suffix}"
    document.metadata = {"source": source, "document_name": document_name}

print(f"# of documents loaded (pre-chunking) = {len(documents)}")

Verify document metadata

In [ ]:
documents[0].metadata

## Chunk documents - TextSplitter

Split the documents to smaller chunks. When splitting the document, ensure a few chunks can fit within the context length of LLM.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
)
doc_splits = text_splitter.split_documents(documents)

# Add chunk number to metadata
for idx, split in enumerate(doc_splits):
    split.metadata["chunk"] = idx

print(f"# of documents = {len(doc_splits)}")

In [ ]:
doc_splits[0].metadata

## Configure `VertexFeatureStore` as Vector Store

You are now ready to start using Vertex Feature Store! 
You can initialize the class by providing `project_id`, `location`, a BQ `dataset_name` and `table_name` to be used to store embeddings. 
You can also point to an existing table. By default the class will use [BigQuery Vector Search](https://cloud.google.com/bigquery/docs/vector-search-intro) to perform vector search.

See [here](TODO) for the full list of parameters of the class. 

In [ ]:
store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    location=REGION,
    dataset_name=DATASET+"13",
    table_name=TABLE+"2333",
    embedding=embedding_model,
 )

In [32]:
store.add_documents(doc_splits)

Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync Succeed for projects/cloud-llm-preview2/locat

TimeoutExpired: Command 'Timeout of 6000 seconds exceeded' timed out after 6000 seconds

In [33]:
## Now using BQ search
store.similarity_search("test",k=6)

ServiceUnavailable: 503 DNS resolution failed for :443: unparseable host:port

In [ ]:
store = store.get_vertex_fs_vector_store()

In [ ]:
## Now using FS search
store.similarity_search("test",k=6)

In [ ]:
vertex_fs.add_documents(doc_splits)

In [ ]:
from google.cloud.aiplatform_v1beta1 import NearestNeighborQuery




In [ ]:
%%time

vertex_fs.similarity_search(
            "treat",
            k=6,
            # \string_filters=[
            #    NearestNeighborQuery.StringFilter({"name":"kind","deny_tokens":["treat"]})
            # ]
        )

Verify the BigQueryVectorSearch with similarity search

### Get a langchain retriever
The retriever will be used in a Langchain Chain to find the most similar documents for a given query.

In [ ]:
langchain_retriever = vertex_fs.as_retriever()

### Compose a Langchain Chain

We are going to use the [`RetrievalQA` chain](https://python.langchain.com/docs/modules/chains/popular/vector_db_qa)
There are several different chain types available, listed [here](https://docs.langchain.com/docs/components/chains/index_related_chains).

In [ ]:
%%time
from langchain_google_vertexai import VertexAI
from langchain.chains import RetrievalQA
from langchain.globals import set_debug

# Set high verbosity
set_debug(True)

llm = VertexAI(model_name="gemini-pro")

search_query = "What should I do when call the emergency roadside assistance?"  # @param {type:"string"}

retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=langchain_retriever
)
response = retrieval_qa.invoke(search_query)
print("\n################ Final Answer ################\n")
print(response["result"])

## Low latency Vector Search with FeatureStore

We are now ready to perform low latency serving with Feature Store! 

To do that, you can simply use the method `set_executor`, to `feature_online_store` type. 

See the [function definition](TODO) for all the parameters you can use.

In [ ]:
vertex_fs.set_executor({"type": "feature_online_store"})

#### Kick off a synchronization process

You can use the method `sync` to synchronize the data from BigQuery to the Feature Online Store, to achieve low latency serving.
When in a production environment, you can also use `cron_schedule` to setup an automatic scheduled synchronization. 

The synchronization process will take around ~20 minutes. 

In [ ]:
vertex_fs.sync()

You can also monitor the synchronization process from GCP Console: [Vertex AI Feature Store Tab](https://console.cloud.google.com/vertex-ai/feature-store/online-stores)

#### Serve with Feature Online Store

You are now ready to serve with Feature Store! You can re-use the same retriever to perform low-latency Vector Search.

In [ ]:
results = langchain_retriever.invoke(search_query)
results[0]

In [ ]:
%%time
results = langchain_retriever.invoke("Leaks under the vehicle")

In [ ]:
%%time
response = retrieval_qa.invoke(search_query)
print("\n################ Final Answer ################\n")
print(response["result"])

### Filtering by metadata


# Appendix

We add here other useful examples to work with the `VertexFeatureStore` Langchain integration.

### Local Bruteforce

You can also prototype by using a (local) bruteforce executor. During initialization, data is downloaded from BQ to your memory.

You can use it for prototyping when the number of documents is low. 

### Get documents by ID

You can also use the function `get_documents` to retrieve a set of documents given a document ID:


In [ ]:
vertex_fs.get_documents(ids=["65a6b83ec5ae428ab8a9607a4f845e47"])